In [2]:
import torch 
import torch.nn as nn
import numpy as np 

In [25]:
class GraphConvolution(nn.Module):
    def __init__(self,f1,f2,bias=True):
        super(GraphConvolution, self).__init__()
        self.W = nn.Linear(f1,f2)
        self.leakyrelu = nn.LeakyReLU(0.2)
    
        self.f1 = f1 
        self.f2 = f2
        self.bn = nn.BatchNorm2d(1296)
        
    def forward(self, input, adj):
        # input : b,v,t,f1 
        b,v,t, f1 = input.size()
        input = input.reshape(b,v,-1)
        support = self.W(input)  # ( b,v,t*f1 ),  ( t*f1,  t*f2 )-> (b,v,t*f2)
        output = torch.bmm(adj.repeat(b,1,1), support)    # (v,v) * ( b, v,t*f2 ) -> (b , v , t*f2)
        output = output.reshape(b,v,t,-1)
        output = self.bn(output)
        return self.leakyrelu(output)

In [26]:
# 三张图加在一起
class GCN(torch.nn.Module):   
    def __init__(self,f1, f2 ): 
        super(GCN, self).__init__() 
        self.cn1 = GraphConvolution(f1,f2)
        self.cn2 = GraphConvolution(f1,f2)
        self.cn3 = GraphConvolution(f1,f2)
        
    def forward(self, x, graph):
        output = self.cn1(x, graph[0])+self.cn2(x,graph[1])+self.cn3(x,graph[2])
        return output

In [27]:
# test gcn
gcn = GCN(1*6,2*6)
d = torch.rand(32, 1296, 6, 1)
g = torch.rand(3,1296,1296)
gcn(d,g).shape 

torch.Size([32, 1296, 6, 2])

In [30]:
'ss'.startswith('s')

True

In [31]:
abs(-1)

1